In [16]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [17]:
with open('exp.txt', 'r') as file:
    lines = file.readlines()

data = []

column_names = lines[0].strip().split(' ')

# 在列名列表的开头插入额外的列名 "gene"
column_names.insert(0, 'Gene')

# 将处理后的列名添加到数据
data.append(column_names)

# 从第二行遍历每一行数据
for line in lines[1:]:
    # 去除行末的换行符并拆分数据
    line_data = line.strip().split(' ')
    # 将拆分后的数据添加到列表中
    data.append(line_data)

# 创建一个 Pandas DataFrame，并使用列名
import pandas as pd

# 将列表转换为 Pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# 最后，将 DataFrame 保存为 CSV 文件
df.to_csv('exp_file.csv', index=False)

In [18]:
with open('methy.txt', 'r') as file:
    lines = file.readlines()

data = []

column_names = lines[0].strip().split(' ')

# 在列名列表的开头插入额外的列名 "DNA"
column_names.insert(0, 'DNA')

# 将处理后的列名添加到数据
data.append(column_names)

# 从第二行遍历每一行数据
for line in lines[1:]:
    # 去除行末的换行符并拆分数据
    line_data = line.strip().split(' ')
    # 将拆分后的数据添加到列表中
    data.append(line_data)

# 创建一个 Pandas DataFrame，并使用列名
import pandas as pd

df = pd.DataFrame(data[1:], columns=data[0])

# 最后，将 DataFrame 保存为 CSV 文件
df.to_csv('methy_file.csv', index=False)

In [19]:
with open('mirna.txt', 'r') as file:
    lines = file.readlines()

data = []

column_names = lines[0].strip().split(' ')

# 在列名列表的开头插入额外的列名 "miRNA"
column_names.insert(0, 'miRNA')

# 将处理后的列名添加到数据
data.append(column_names)

# 从第二行遍历每一行数据
for line in lines[1:]:
    # 去除行末的换行符并拆分数据
    line_data = line.strip().split(' ')
    # 将拆分后的数据添加到列表中
    data.append(line_data)

# 创建一个 Pandas DataFrame，并使用列名
import pandas as pd

df = pd.DataFrame(data[1:], columns=data[0])

# 最后，将 DataFrame 保存为 CSV 文件
df.to_csv('mirna_file.csv', index=False)

In [20]:
exp = pd.read_csv('exp_file.csv')
methy=pd.read_csv('methy_file.csv')
mirna=pd.read_csv('mirna_file.csv')

In [21]:
# 定义一个函数，用于将列名中的 "-" 替换为 "."
def replace_dash_with_dot(column_name):
    return column_name.replace('-', '.')

In [22]:
# 使用 rename() 方法替换列名
methy.rename(columns=replace_dash_with_dot, inplace=True)

In [8]:
exp_columns = set(exp.columns)
methy_columns = set(methy.columns)
mirna_columns = set(mirna.columns)
# exp & methy & mirna相同相同column名字的列数 每一个column代表一名病人 也就是3个omics data记录的common病人数量
common_columns_count = len(exp_columns & methy_columns & mirna_columns)
common_columns_count

291

In [9]:
common_columns = exp.columns.intersection(methy.columns).intersection(mirna.columns)
# 选择这些列
exp_common = exp[common_columns]
methy_common = methy[common_columns]
mirna_common = mirna[common_columns]

In [10]:
multi_data = pd.concat([exp_common, methy_common, mirna_common], axis=0, ignore_index=True)

In [11]:
#normalization
standard_scaler = StandardScaler()
multi_data_standardized = standard_scaler.fit_transform(multi_data)

In [12]:
print(multi_data.shape)

(26236, 291)


In [13]:
#置换行和列，将数据的特征（列）和样本（行）重新排列
tcga_input=np.transpose(multi_data_standardized)
print(tcga_input.shape)

(291, 26236)


In [14]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler


# 定义神经网络的隐藏层的大小（神经元的数量）的参数
n_hidden_1 = 200
n_hidden_2 = 100
n_hidden_3 = 50

# 将 TensorFlow 切换回 graph execution 模式
tf.compat.v1.disable_eager_execution()

# 输入数据的特征数量
n_input = tcga_input.shape[1]


# 创建输入占位符，维度设置为 (None, n_input)
X = tf.compat.v1.placeholder("float", [None, n_input])

# 神经网络的权重矩阵和偏置项
weights = {
    'encoder_h1': tf.Variable(tf.random.normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'encoder_h3': tf.Variable(tf.random.normal([n_hidden_2, n_hidden_3])),
    'decoder_h1': tf.Variable(tf.random.normal([n_hidden_3, n_hidden_2])),
    'decoder_h2': tf.Variable(tf.random.normal([n_hidden_2, n_hidden_1])),
    'decoder_h3': tf.Variable(tf.random.normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random.normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random.normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random.normal([n_hidden_3])),
    'decoder_b1': tf.Variable(tf.random.normal([n_hidden_2])),
    'decoder_b2': tf.Variable(tf.random.normal([n_hidden_1])),
    'decoder_b3': tf.Variable(tf.random.normal([n_input])),
}

# 编码器函数---------如果是DAE后面要加biases['encoder_b1']，这是和你的autoencoder不一样的地方，你可以自己对比一下代码
def encoder(x):
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['encoder_h3']), biases['encoder_b3']))
    return layer_3

# 解码器函数
def decoder(x):
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['decoder_h3']), biases['decoder_b3']))
    return layer_3


In [15]:
# 定义 DAE 的输入和输出

noise_factor = 0.1  # 引入的噪声因子

# 使用 tf.random.normal 生成随机噪声，维度与 multi_data_standardized 相同
noisy_data = tcga_input + noise_factor * tf.random.normal(shape=tf.shape(tcga_input), mean=0.0, stddev=1.0, dtype=tf.float32)

# 编码器和解码器的输出
encoded = encoder(noisy_data)
decoded = decoder(encoded)

# 损失函数：均方差
cost = tf.reduce_mean(tf.square(X - decoded))

# 优化器
learning_rate = 0.01

# 获取编码器和解码器的可训练变量
encoder_trainable_variables = list(weights.values()) + list(biases.values())
decoder_trainable_variables = list(weights.values()) + list(biases.values())

# 将编码器和解码器的可训练变量合并为一个列表
trainable_variables = encoder_trainable_variables + decoder_trainable_variables

# 创建优化器
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)

# 使用优化器最小化损失
train_op = optimizer.minimize(cost, var_list=trainable_variables)

# 初始化 TensorFlow 变量
init = tf.compat.v1.global_variables_initializer()

# 定义训练步骤
def train_step():
    _, c = sess.run([train_op, cost], feed_dict={X: noisy_data.eval()})
    return c

# 设置训练周期数量和显示步骤
num_epochs = 1000
display_step = 100

# 运行训练步骤
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        train_loss = train_step()
        if epoch % display_step == 0:
            print(f"Epoch {epoch}, Loss: {train_loss:.4f}")

2023-09-06 18:35:21.640946: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Epoch 0, Loss: 1.9496
Epoch 100, Loss: 1.5310
Epoch 200, Loss: 1.5316
Epoch 300, Loss: 1.5328
Epoch 400, Loss: 1.5334
Epoch 500, Loss: 1.5331
Epoch 600, Loss: 1.5340
Epoch 700, Loss: 1.5341
Epoch 800, Loss: 1.5344
Epoch 900, Loss: 1.5344


In [25]:
decoded

Tensor("Tanh_5:0", shape=(291, 26236), dtype=float32)
